# Wikipedia-searching agent

Python 3.11

In [ ]:
!pip install chromadb instructor wikipedia

In [2]:
import instructor
import pydantic
import wikipedia

In [3]:
# OpenAI setup

import os

from dotenv import load_dotenv
from openai import OpenAI


load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OpenAI API key not found")

client = OpenAI(api_key=openai_api_key)

def openai_request(prompt, system=None, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system or "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

## Calculator

In [4]:
# Generate execution plan

question = "How much is fifty five plus eleven minus twenty two?"  # 55 + 11 - 22
system = """You are a helpful assistant that uses its tools to solve problems. 

Help the user to create an algorithm as a list of steps that have to be performed in order to answer to the question. 

While plannninng the steps, you can only use the following tools:
- calculator - while formulating a step using this tool, you should choose a single specific opeartion (eg. addition) and name the values that it should operate on. If the values proceed from previous steps, name the step in which the value was obtained. For example, you can phrase the step as: ```add the value from step 1 to the value from step 3.```. If the maht operation requires using several operations, phrase them all as separate steps.

Use specific terms and give concise instructions. The number of steps should be as low as possible. Put the steps in separate lines and number them.
"""
prompt = f"Create a list of steps that have to be performed in order to answer to the question: ```{question}```"

response = openai_request(prompt=prompt, system=system)
response  #  '1. Add 2 and 2.\n2. Subtract 1 from the result of step 1.'

'1. Add fifty five and eleven.\n2. Subtract twenty two from the result.'

In [38]:
# Extract list of steps and their details with Instructor package

from enum import Enum

from typing import Union
from pydantic import BaseModel


class Step(BaseModel):
    step_number: int

class ResultOfPreviousStep(BaseModel):
    number_of_step: int

class CalculatorOpeartion(str, Enum):
    Add = "Add"
    Subtract = "Subtract"
    Multiply = "Multiply"
    Divide = "Divide"

class CalculatorStep(Step):
    operation: CalculatorOpeartion
    first_value: Union[int, float, ResultOfPreviousStep] # fails at order of values
    second_value: Union[int, float, ResultOfPreviousStep]

STEP_TYPES = [CalculatorStep]

class StepsPlan(BaseModel):
    steps_list: list[Union[*STEP_TYPES]]

instructor_client = instructor.from_openai(OpenAI())

def create_plan(data: str) -> StepsPlan:
    return instructor_client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=StepsPlan,
        messages=[
            {
                "role": "user",
                "content": f"Convert the following list of steps to a plan: {data}.",
            },
        ],
    )

plan = create_plan(response)
for step in plan.steps_list:
    print(type(step), step) 
# <class '__main__.CalculatorStep'> step_number=1 operation=<CalculatorOpeartion.Add: 'Add'> first_value=55 second_value=11
# <class '__main__.CalculatorStep'> step_number=2 operation=<CalculatorOpeartion.Subtract: 'Subtract'> first_value=22 second_value=ResultOfPreviousStep(number_of_step=1) # wrong order of values

<class '__main__.CalculatorStep'> step_number=1 operation=<CalculatorOpeartion.Add: 'Add'> first_value=55 second_value=11
<class '__main__.CalculatorStep'> step_number=2 operation=<CalculatorOpeartion.Subtract: 'Subtract'> first_value=22 second_value=ResultOfPreviousStep(number_of_step=1)


In [39]:
# Execute the plan with agents

from abc import ABC, abstractmethod
from dataclasses import dataclass


class Agent(ABC):
    @classmethod
    @abstractmethod
    def execute(cls):
        pass

@dataclass
class CalculatorInput:
    operation: CalculatorOpeartion
    value1: Union[int, float]
    value2: Union[int, float]
    
class CalculatorAgent(Agent):
    @classmethod
    def _calculate(cls, input: CalculatorInput) -> Union[int, float]:
        if input.operation == CalculatorOpeartion.Add:
            return input.value1 + input.value2
        elif input.operation == CalculatorOpeartion.Subtract:
            return input.value1 - input.value2
        elif input.operation == CalculatorOpeartion.Multiply:
            return input.value1 * input.value2
        else:
            return input.value1 / input.value2
    
    @classmethod
    def execute(cls, input: CalculatorInput) -> Union[int, float]:
        return CalculatorAgent._calculate(input)


def execute_plan(plan: StepsPlan) -> dict:
    execution = {}
    for i, step in enumerate(plan.steps_list, 1):
        if isinstance(step, CalculatorStep):
            input = CalculatorInput(
                operation=step.operation,
                value1=execution[step.first_value.number_of_step] if isinstance(step.first_value, ResultOfPreviousStep) else step.first_value,
                value2=execution[step.second_value.number_of_step] if isinstance(step.second_value, ResultOfPreviousStep) else step.second_value,
            )
            result = CalculatorAgent.execute(input)
        execution[i] = result
    return execution

execute_plan(plan)

{1: 66, 2: -44}

## Wikipedia

In [ ]:
# Generate execution plan

# question = "What is the time difference between Lisbon and Buenos Aires?"
question = "What is the time difference between Lisbon and Buenos Aires?"
system = """You are a helpful assistant that uses its tools to solve problems. 

Help the user to create an algorithm as a list of steps that have to be performed in order to answer to the question. 

While plannninng the steps, you can only use the following tools:
- wikipedia search - you can phrase the step as: search wikipedia page of topic,
- calculator - should name the specific calculation operator, eg. subtract or add. If the values proceed from previous steps, name the step in which the valye was obtained. For example, you can phrase the step as: ```subtract the value from step from the value from step 3.```.

Use specific terms and give concise instructions. The number of steps should be as low as possible. Put the steps in separate lines and number them.
"""
prompt = f"Create a list of steps that have to be performed in order to answer to the question: ```{question}```"

response = openai_request(prompt=prompt, system=system)
response  # '1. Search Wikipedia page for current local time in Lisbon.\n2. Search Wikipedia page for current local time in Buenos Aires.\n3. Calculate the time difference between the two cities by subtracting the time in Lisbon from the time in Buenos Aires.'

In [ ]:
# Extract list of steps and their details with Instructor package

from typing import Union
from pydantic import BaseModel


class Step(BaseModel):
    step_number: int

class WikipediaStep(Step):
    wikipedia_page: str
    what_to_find: str

class CalculatorStep(Step):
    operation: str
    value1: str
    value2: str

# STEP_TYPES = [Wikipedia, Calculator]

class StepsPlan(BaseModel):
    steps_list: list[Union[WikipediaStep, CalculatorStep]]

instructor_client = instructor.from_openai(OpenAI())

def create_plan(data: str) -> StepsPlan:
    return instructor_client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=StepsPlan,
        messages=[
            {
                "role": "user",
                "content": f"Convert the following list of steps to a plan: {data}",
            },
        ],
    )

classification = list(create_plan(response))
classification
# [('steps_list',
#   [WikipediaStep(step_number=1, wikipedia_page='Lisbon', what_to_find='current local time'),
#    WikipediaStep(step_number=2, wikipedia_page='Buenos Aires', what_to_find='current local time'),
#    CalculatorStep(step_number=3, operation='subtract', value1='Lisbon', value2='Buenos Aires')])]

In [152]:
wiki_step1, wiki_step2, calc_step = classification[0][1]
wiki_step1

WikipediaStep(step_number=1, wikipedia_page='Lisbon', what_to_find='current local time')

In [158]:
# Get wikipedia page and search by item similarity
def get_wikipedia_page_html(wikipedia_query: WikipediaStep) -> str:
    page = wikipedia.page(wikipedia_query.wikipedia_page)
    return page.html()

def create_vector_db(html_content: str) -> str:
    return "Vector database"


page = get_wikipedia_page_html(wiki_step1)

In [ ]:
page.html()